In [1]:
from load import ModelLoader
import torch
from transformers import AutoModel, MistralModel


/Users/alialh/Documents/AI Research/Merging/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_names = ["Open-Orca/Mistral-7B-OpenOrca", "argilla/CapybaraHermes-2.5-Mistral-7B"]
model_weights = [1, 0.5]
model_loader = ModelLoader(model_names, model_weights)

sum_weights = sum(model_weights)
normalized_weights = [weight/sum_weights for weight in model_weights]

/Users/alialh/Documents/AI Research/Merging/.venv/lib/python3.12/site-packages/accelerate/utils/modeling.py:1365: UserWarning: Current model requires 905976576 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:46<00:00, 23.31s/it]
/Users/alialh/Documents/AI Research/Merging/.venv/lib/python3.12/site-packages/accelerate/utils/modeling.py:1365: UserWarning: Current model requires 1040195328 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 3/3 [00:06<00:00,  2.25s/it]


In [ ]:
def print_model_params(model: MistralModel):
  for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()}")


In [ ]:
# Assume `model` is your instantiated Mistral model
device = torch.device("mps" if torch.has_mps else "cpu")

In [3]:
for layer_1, layer_2 in zip(*[model.layers for model in model_loader.models]):
  for n_1, n_2 in zip(layer_1.named_parameters(), layer_2.named_parameters()):
    name_1, param_1 = n_1
    name_2, param_2 = n_2

    print(name_1, "  ", name_2)

    # param_1 = param_1.to(device)
    # param_2 = param_2.to(device)

    tensors = [param_1.data, param_2.data]
    tensors = torch.stack(tensors, dim=0)
    weights = torch.tensor(normalized_weights, dtype=tensors.dtype, device=tensors.device)

    while len(weights.shape) < len(tensors.shape):
      weights.unsqueeze_(-1)

    res = (weights * tensors).sum(dim=0)
    # res = normalized_weights[0]*param_1.data + normalized_weights[1]*param_2.data

    print(res)

  print("\n")

self_attn.q_proj.weight    self_attn.q_proj.weight
tensor([[ 6.2565e-05,  1.2267e-03, -3.5445e-05,  ...,  4.3996e-03,
          2.4100e-05, -6.2227e-04],
        [-5.8711e-05, -3.5731e-04,  2.6623e-05,  ..., -2.7831e-04,
          3.6756e-06,  3.2393e-04],
        [ 1.8378e-05, -5.1943e-04, -1.7087e-06,  ..., -6.0755e-03,
         -1.7881e-05, -4.1326e-04],
        ...,
        [-2.4398e-05, -1.7211e-03, -1.1563e-05,  ...,  4.3511e-04,
          7.9473e-06,  3.5254e-04],
        [ 7.4903e-06, -1.5564e-03,  1.8358e-05,  ..., -2.0580e-03,
         -3.2167e-05,  7.6898e-04],
        [ 2.6286e-05,  1.9836e-03,  6.6559e-06,  ..., -2.3548e-04,
         -8.0069e-06, -4.9480e-04]], device='mps:0')
self_attn.k_proj.weight    self_attn.k_proj.weight
tensor([[ 4.3611e-06, -3.3436e-03,  1.3244e-04,  ..., -1.6363e-02,
          1.8891e-04, -8.7134e-04],
        [ 2.3444e-06,  1.9223e-03, -8.9149e-05,  ...,  7.0178e-03,
         -9.7732e-05,  8.3033e-04],
        [-1.8497e-05,  3.0518e-03, -6.7612e-

RuntimeError: Tensor on device meta is not on the expected device mps:0!

In [ ]:
model_dict = model_loader.models[0].state_dict()

In [ ]:
for name, module in model_loader.models[0].named_modules():
  print(name)

In [ ]:
for t_1, t_2 in zip(*[model.named_parameters() for model in model_loader.models]):
  name_1, param_1 = t_1
  name_2, param_2 = t_2
  print(f"Layer1: {name_1} | Size1: {param_1.size()}")
  print(f"Layer2: {name_2} | Size2: {param_2.size()}")
  print("\n")


In [ ]:
print_model_layers(model_loader.models[0])